In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from IPython.display import Image, display  


# Configurações do servidor SMTP
smtp_server = "smtp.gmail.com"
smtp_port = 587

# Suas credenciais
email = "vitoracavaco40@gmail.com"  # Substitua pelo seu e-mail
senha = ""  # Substitua pela sua senha (ou senha de app, se tiver 2FA ativado)

# Caminho para o chromedriver
driver_path = r"C:\Users\Vitor\Documents\chromedriver-win64\chromedriver.exe"

# Crie uma instância de Service passando o caminho do driver
service = Service(driver_path)

# Inicie o ChromeDriver com o serviço e as opções de download
driver = webdriver.Chrome(service=service)

print("Robo iniciando pesquisas......\n")

# Acessa o site
driver.get("https://lista.mercadolivre.com.br/celulares-telefones/celulares-smartphones/iphone/linha-iphone-16/iphone-16/novo/iphone-16_NoIndex_True#applied_filter_id%3DITEM_CONDITION%26applied_filter_name%3DCondi%C3%A7%C3%A3o%26applied_filter_order%3D4%26applied_value_id%3D2230284%26applied_value_name%3DNovo%26applied_value_order%3D1%26applied_value_results%3D17%26is_custom%3Dfalse")

#aumenta a tela
driver.maximize_window()

#tempo de espera
wait = WebDriverWait(driver, 40)

#listas os arrays
array_preco_ml = []

array_link_ml = []

array_imagem_ml = []

array_preco_amz = []

array_link_amz = []

array_imagem_amz = []

#Verifica todos os anúncios
lista_iphone_ml = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//a[contains(@class, "poly-component__title")]')))

#Verifica o preço 
preco_iphone_ml  = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//div[contains(@class, "poly-price__current")]//span[contains(@class, "andes-money-amount__fraction")]')))

#Traz a url da imagem
imagem_iphone_ml = wait.until(EC.visibility_of_all_elements_located((By.XPATH,'//div[contains(@class, "poly-card__portada")]//img[contains(@class, "poly-component__picture")]')))

#Traz o link do produto
link_iphone_ml  = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//h3[contains(@class, "poly-component__title-wrapper")]//a[contains(@class, "poly-component__title")]')))

print("--------- Mercado Livre ---------\n")

#Percorre todos os anuncios de iphone
for valor, lista in enumerate(lista_iphone_ml):

    #Converte para texto a listagem (Nome)
    texto = lista.text

    #Converte para float
    preco = float(preco_iphone_ml[valor].text)

    #Rola a tela até o elemento
    driver.execute_script("arguments[0].scrollIntoView(true);", imagem_iphone_ml[valor])

    #Consegue o atributo src da imagem
    imagem = imagem_iphone_ml[valor].get_attribute("src")

    #Consegue o atributo href do link 
    link  = link_iphone_ml[valor].get_attribute("href")

    print(f"{texto}  - R$: {preco} - Link: {link} ")
    
    #Converte a url para mostrar a imagem no display do jupyter
    display(Image(url=imagem, width=150, height=150))
    print("\n")

    #Inserindo valores no array se bater a condição
    if (preco == 5.698):

        array_preco_ml.append(preco)
        array_link_ml.append(link)
        array_imagem_ml.append(imagem)

print('\n-----------------------------------\n')

#Acessa a amazon 
driver.get("https://www.amazon.com.br/ref=nav_logo")

#Faz a pesquisa do produto
pesquisa_amazon = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="twotabsearchtextbox"]')))
pesquisa_amazon.send_keys("Iphone 16")
pesquisa_amazon.send_keys(Keys.RETURN)

#Verifica todos os anúncios
lista_iphone_amz = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//h2[contains(@class, "a-size-base-plus a-spacing-none a-color-base a-text-normal")]')))

#Verifica o preço 
preco_iphone_amz = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//span[contains(@class, "a-price-whole")]')))

#Traz a url da imagem
imagem_iphone_amz = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//div[contains(@class, "a-section aok-relative s-image-square-aspect")]//img[contains(@class,"s-image")]')))

#Traz o link do produto
link_iphone_amz = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//a[contains(@class, "a-link-normal s-line-clamp-4 s-link-style a-text-normal")]')))

print("--------- Amazon ---------\n")

#Procura os primeiros oito resultados
for i in range (8):
 
    texto = (lista_iphone_amz[i].text).upper()

    #Procura a palavra iphone 16, se tiver..
    if "IPHONE 16" in texto:

         #Converte para float
        preco = float(preco_iphone_amz[i].text)

        #Consegue o atributo src da imagem
        imagem = imagem_iphone_amz[i].get_attribute("src")

         #Consegue o atributo href do link 
        link  = link_iphone_amz[i].get_attribute("href")

        print(f"{texto}  - R$: {preco} - Link: {link} \n")
        
        #Converte a url para mostrar a imagem no display do jupyter
        display(Image(url=imagem, width=150, height=150))

        print("\n")

        #Inserindo valores no array se bater a condição
        if (preco == 5.698):

            array_preco_amz.append(preco)
            array_link_amz.append(link)
            array_imagem_amz.append(imagem)

#Fecha o navegador
driver.quit()

#Se tiver produto em promação no ml faça
if array_preco_ml:

  # Criação da mensagem
    msg = MIMEMultipart()
    msg['From'] = email
    msg['To'] = "vitoracavaco40@gmail.com"  # Substitua pelo e-mail do destinatário
    msg['Subject'] = "Promoção Iphone"

    #valores = ", ".join(map(str,array_preco_ml))
    
    # Corpo do e-mail

    corpo_email = f"Promoção do Iphone 16 no mercado livre\n\n"

    for preco, link, imagem in zip(array_preco_ml, array_link_ml, array_imagem_ml):
        corpo_email += f"- Preço: R$ {preco}\n\n  Link: {link}\n\n  Imagem: {imagem}\n\n"

    msg.attach(MIMEText(corpo_email, 'plain'))

    # Conectar ao servidor SMTP e enviar o e-mail
    try:
        # Estabelecendo conexão com o servidor SMsTP
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Ativa a criptografia TLS para segurança

        # Login na conta
        server.login(email, senha)

        # Enviar o e-mail
        server.sendmail(msg['From'], msg['To'], msg.as_string())

        print("E-mail enviado com sucesso!")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

    finally:
        # Fechar a conexão
        server.quit()

#Se tiver produto em promação no ml faça
if array_preco_amz:
    
  # Criação da mensagem
    msg = MIMEMultipart()
    msg['From'] = email
    msg['To'] = "vitoracavaco40@gmail.com"  # Substitua pelo e-mail do destinatário
    msg['Subject'] = "Promoção Iphone"

    #valores = ", ".join(map(str, array_preco_amz))

    # Corpo do e-mail
    corpo_email = f"Promoção do Iphone 16 na amazon\n\n"

    for preco, link, imagem in zip(array_preco_amz, array_link_amz, array_imagem_amz):
        corpo_email += f"- Preço: R$ {preco}\n\n  Link: {link}\n\n  Imagem: {imagem}\n\n"

    msg.attach(MIMEText(corpo_email, 'plain'))

    # Conectar ao servidor SMTP e enviar o e-mail
    try:
        # Estabelecendo conexão com o servidor SMTP
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Ativa a criptografia TLS para segurança

        # Login na conta
        server.login(email, senha)

        # Enviar o e-mail
        server.sendmail(msg['From'], msg['To'], msg.as_string())

        print("E-mail enviado com sucesso!")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

    finally:
        # Fechar a conexão
        server.quit()        
